
# Uma breve introdução ao Machine Learning
## Problemas propostos #04

- **Dia 4:** [Medição e redução de dimensionalidade.](https://github.com/GabrielWendell/Intro_ML/blob/main/Notebooks/Dia4.ipynb)

---

### Você vai precisar...

In [ ]:
from mls import locate_data
spectra_data = pd.read_hdf(locate_data('spectra_data.hf5'))

---

## Problema 1

**a)** O [algoritmo Expectation-Maximization (EM)](https://en.wikipedia.org/wiki/Expectation–maximization_algorithm) é usado para implementar muitos métodos de aprendizado de máquina, incluindo K-means, análise fatorial e PCA ponderado.

A ideia básica do EM é otimizar uma função objetivo que depende de dois conjuntos disjuntos de parâmetros, atualizando alternadamente um conjunto e depois o outro, usando um esquema que garante a melhoria da função objetivo (embora geralmente para um ótimo local em vez de global. ). As atualizações alternadas são chamadas de E-step e M-step.

O K-means é um dos usos mais simples do EM, portanto, é uma boa maneira de obter alguma experiência prática.

Implemente a função abaixo para executar uma etapa E K-means.

- *Dica:* você pode achar o [`np.argmin`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argmin.html) útil.

O objetivo deste exercício é escrever uma solução correta, em vez do código mais rápido possível. Portanto, sinta-se à vontade para usar loops sobre índices de matriz, pois geralmente são mais fáceis de entender e acertar do que o código numpy vetorizado equivalente. 

(Você terá alguma prática escrevendo código vetorizado na próxima pergunta).

In [ ]:
def E_step(X, centers):
    """"Execute um passo E K-means."
    
    Atribua cada amostra ao cluster com o centro mais próximo, usando o
    Norma euclidiana para medir a distância entre uma amostra e um centro de cluster.
    
    Parâmetros
    ----------
    X : array com shape (N, D)
        Dados de entrada consistindo de N amostras em D dimensões.
    centers : array com shape (n, D)
        Centros dos n clusters em D dimensões.
        
    Retorno
    -------
    integer array com shape (N,)
        Índice do cluster de cada sample, no intervalo 0 para n - 1
    """
    N, D = X.shape
    n = len(centers)
    
    assert centers.shape[1] == D
    indexes = np.empty(N, int)
    
    # SEU CÓDIGO AQUI
    
    raise NotImplementedError()
    
    return indexes

**b)** Efetue os testes de validação abaixo:

In [ ]:
gen = np.random.RandomState(seed=123)
X = gen.normal(size=(20, 2))
centers = np.array([[0., 0.], [0., 10.]])
X[50:] += centers[1]
indices = E_step(X, centers)

assert np.all(indices[:50] == 0)
assert np.all(indices[50:] == 1)

centers = gen.uniform(size=(5, 2))
indexes = E_step(X, centers)

assert np.array_equal(indexes, [4, 1, 4, 4, 1, 0, 1, 0, 2, 1, 2, 4, 0, 1, 0, 1, 0, 1, 4, 4])

**c)** Implemente a função abaixo para executar uma etapa M de K-means:

In [ ]:
def M_step(X, indexes, n):
    """Perform a K-means M-step.
    
    Calcule o centro de cada cluster como a média de suas amostras atribuídas.
    
    Os centros de quaisquer clusters sem quaisquer amostras atribuídas devem ser definidos para a origem.
    
    Parâmetros
    ----------
    X : array com shape (N, D)
        Dados de entrada consistindo de N amostras em D dimensões.
        
    indexes : integer array com shape (N,)
        Índices do cluster de cada sample, no intervalo de 0 até n - 1
        
    n : int
        Number de clusters.  Deve ser <= N.
        
    Retorno
    -------
    array com shape (n, D)
        Centros dos n clusters em D dimensões.
    """
    N, D = X.shape
    assert indices.shape == (N,)
    assert n <= N
    centers = np.zeros((n, D))
    
    # SEU CÓDIGO AQUI
    
    raise NotImplementedError()
    
    return centers

**d)**  Aplique os *sanity tests* abaixo:

In [ ]:
gen = np.random.RandomState(seed=123)
X = np.ones((20, 2))
indices = np.zeros(20, int)
centers = M_step(X, indices, 1)

assert np.all(centers == 1.)

################################

n = 5
indices = gen.randint(n, size=len(X))
centers = M_step(X, indices, n)

assert np.all(centers == 1.)

################################

X = gen.uniform(size=X.shape)
centers = M_step(X, indices, n)

assert np.allclose(
    np.round(centers, 3),
    [[ 0.494,  0.381], [ 0.592,  0.645],
     [ 0.571,  0.371], [ 0.234,  0.634],
     [ 0.250,  0.386]])

**e)** Agora você implementou o núcleo do algoritmo K-means. Experimente com este *wrapper* simples, que faz um gráfico de dispersão das duas primeiras colunas após cada iteração. O wrapper sklearn combina o resultado de vários pontos de partida aleatórios e possui outros refinamentos.

In [ ]:
def KMeans_fit(data, n_clusters, nsteps, seed = 123):
    X = data.values
    N, D = X.shape
    assert n_clusters <= N
    gen = np.random.RandomState(seed=seed)
    
    # Escolha amostras aleatórias como os centros iniciais.
    shuffle = gen.permutation(N)
    centers = X[shuffle[:n_clusters]]
    
    # Execute uma etapa E inicial para atribuir amostras a clusters.
    indices = E_step(X, centers)
    
    # Faça um loop sobre as iterações.
    for i in range(nsteps):
        centers = M_step(X, indices, n_clusters)
        indices = E_step(X, centers)
        
    # Plote o resultado
    cmap = np.array(sns.color_palette())
    plt.scatter(X[:, 0], X[:, 1], c=cmap[indices % len(cmap)])
    plt.scatter(centers[:, 0], centers[:, 1], marker='+', c='k', s=400, lw=5)
    opt_plot()
    plt.show()

**e)** Experimente em alguns dados 2D gerados aleatoriamente com 3 *clusters* separados (usando o prático [make_blobs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html)). Para este teste simples, você deve encontrar uma boa solução após duas iterações:

In [ ]:
from sklearn.datasets.samples_generator import make_blobs

gen = np.random.RandomState(seed=123)
X, _ = make_blobs(500, 2, [[-3,-3],[0,3],[3,-3]], random_state=gen)
data = pd.DataFrame(X, columns=('x0', 'x1'))

KMeans_fit(data, n_clusters=3, nsteps=0);
KMeans_fit(data, n_clusters=3, nsteps=1);
KMeans_fit(data, n_clusters=3, nsteps=2);

---

## Problema 2

**a)** O passo E no problema anterior usa a matriz de separação $S_{ij}$ entre o $0\leq i<N$ ($D$-dimensional) *samples* $\textbf{x}_{i}$ e $0\leq i< n$ ($D$- dimensional) *cluster* significa $\mu_{j}$:

$$S_{ij}\equiv|\textbf{x}_{i}-\mu_{j}|^{2},$$

Observe que este é um matriz retangular $N\times n$, então geralmente não é quadrada. 

Implemente a função abaixo para calcular esta matriz usando `PyTorch`, sem loops explícitos sobre índices de tensor.

In [ ]:
import torch

def S_matrix(X, mu):
    """Calcule a matriz de separação usada pela etapa K-médias E.
    
    Calcule a matriz 2D S com elementos |x[i] - mu[j]|^2 usando
    PyTorch e sem loops explícitos.
    
    Parâmetros
    ----------
    X : torch tensor com shape (N, D)
        Dados de entrada consistindo de N amostras em D dimensões.
    mu : torch tensor com shape (n, D)
        Centros dos n clusters em D dimensões.
    
    Retorna
    -------
    array com shape (N, n)
    """
    N, D = X.size()
    n = len(mu)
    assert mu.shape[1] == D
    
    # SEU CÓDIGO AQUI
    
    raise NotImplementedError()

**b)** Aplique os seguintes *sanity tests*:

In [ ]:
X = torch.zeros(5, 3)
mu = torch.ones(2, 3)
S = S_matrix(X, mu)

assert S.size() == (5, 2)
assert torch.allclose(S, torch.tensor(3.))

################################

gen = np.random.RandomState(seed=123)
X = torch.tensor(gen.normal(size=(1000, 2)), dtype=torch.float32)
mu = torch.tensor([[0., 0.], [0., 10.]])
X[50:] += mu[1]
S = S_matrix(X, mu)

assert S.size() == (1000, 2)
assert torch.allclose(torch.mean(S, dim=0), torch.tensor([96.8427, 6.9882]))

---

## Problema 3

**a)** O método PCA de redução de dimensionalidade primeiro calcula uma decomposição linear exata (até o erro de arredondamento), então apara linhas e colunas para o número desejado de variáveis latentes. Neste problema, você explorará como o PCA é implementado. A complicada álgebra linear já está implementada em [`numpy.linalg`](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.linalg.html), mas ainda é um desafio manter todas as notações e convenções auto consistentes.

Os dados de entrada $X$ são fornecidos como uma matriz $N\times D$ (amostras, características). A seguir, assumimos que cada recurso está centrado em zero (caso contrário, calcule e subtraia o $\mu_{j}$ , depois adicione-os de volta mais tarde),

$$\mu_{j}=\sum_{i}X_{ij}=0.$$

Existem três métodos equivalentes para realizar a decomposição exata inicial:

1. Calcular a [matriz de covariância da amostra](https://en.wikipedia.org/wiki/Sample_mean_and_covariance#Sample_covariance) $D\times D$

$$C\equiv\frac{1}{N-1}X^{T}X.$$

Então encontre sua decomposição de autovalor:

$$C=Q^{T}\Lambda Q$$

onde $\Lambda$ é uma matriz diagonal $D\times D$ de autovalores e as linhas da matriz ortogonal $D\times D$ do tipo $Q$ são os autovetores correspondentes.

2. Calcule a matriz $N\times N$ de produtos escalares entre amostras:

$$D\equiv\frac{1}{N-1}XX^{T},$$

então encontre sua decomposição de autovalor, onde agora $Q$ e $\Lambda$ são matrizes $N\times N$.

3. Encontre a [decomposição de valor singular (SVD)](https://en.wikipedia.org/wiki/Singular_value_decomposition) de $X$

$$X=USV\implies C=\frac{1}{N-1}V^{T}S^{2}V$$

onde $S$ é uma matriz diagonal $K\times K$ de valores singulares, $U$ é $N\times K$ e $V$ é $K\times D$, com $K=\min{(N,D)}$. A notação acima é escolhida para corresponder a [`np.linalg.svd`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.svd.html) que você usará abaixo.

O custo computacional de cada método depende diferentemente dos valores de $N$ e $D$, então o método mais eficiente dependerá da forma dos dados de entrada $X$. Há também considerações numéricas: as matrizes $C$ e $D$ devem ser [definidas positivamente](https://en.wikipedia.org/wiki/Definite_matrix) para garantem autovalores positivos, mas isso não será verdade para $C$ se $N<D$ ou para $D$ se $N>D$.

Implemente a função abaixo para calcular os autovetores e autovalores de uma matriz quadrada de entrada usando uma função adequada de [`np.linalg`](https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.linalg.html). Os resultados devem ser classificados em ordem decrescente de autovalores, conforme necessário pelo PCA. 

- **Dica**: M[::-1] inverte as linhas de um array 2D `M`.

In [ ]:
def eigensolve(M):
    """Calcular autovetores e autovalores de uma matriz quadrada simétrica.
    
    Os resultados são classificados por autovalor decrescente. As linhas (não colunas) do
    array de autovetores retornados são os autovetores normalizados de M.
    
    Parâmetros
    ----------
    M : 2D array
        N x N matriz quadrada simétrica para usar.
        
    Retorna
    -------
    tuple
        Tupla de matrizes (valores próprios, vetores próprios) com valores próprios decrescentes e
        autovetor[i] correspondente ao autovalor[i]. Os valores próprios devem ter o
        forma (N,) e vetores próprios devem ter a forma (N, N).
    """
    assert len(M.shape) == 2
    nrows, ncols = M.shape
    
    assert nrows == ncols
    assert np.all(M.T == M)
    
    # SEU CÓDIGO AQUI
    
    raise NotImplementedError()

**b)** Efetue os seguintes testes de validação:

In [ ]:
C = np.diag(np.arange(1., 5.))
evals, evecs = eigensolve(C)

assert np.allclose(
    evals,
    [ 4.,  3.,  2.,  1.])
assert np.allclose(
    evecs,
    [[ 0.,  0.,  0.,  1.],
     [ 0.,  0.,  1.,  0.],
     [ 0.,  1.,  0.,  0.],
     [ 1.,  0.,  0.,  0.]])

################################

gen = np.random.RandomState(seed=123)
N, D = 4, 3
X = gen.uniform(size=(N, D))
X -= np.mean(X, axis=0)
C = np.dot(X.T, X) / (N - 1)
evals, evecs = eigensolve(C)

assert np.allclose(C, evecs.T.dot(np.diag(evals).dot(evecs)))
assert np.allclose(
    np.round(evals, 5),
    [ 0.08825,  0.0481 ,  0.01983])
assert np.allclose(
    np.round(evecs, 3),
    [[-0.787, -0.477,  0.391],
     [-0.117,  0.737,  0.665],
     [-0.606,  0.478, -0.636]])

**c)** Implemente a função abaixo para calcular as mesmas quantidades usando o [método SVD](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.svd.html) diretamente em $X$ em vez de resolver o problema de autovalor para a covariância da amostra. 

- **Dica**: preste atenção no parâmetro `full_matrices`.

In [ ]:
def svdsolve(X):
    """Calcule autovetores e autovalores da covariância amostral de X.

    Os resultados são classificados por autovalor decrescente. As linhas (não colunas) do
    array de autovetores retornados são os autovetores normalizados de M.

    Usa o método SVD diretamente no X.
    
    Parâmetros
    ----------
    X: 2D array
        N x D matriz a ser usada.
        
    Retorna
    -------
    tuple
        Tupla de matrizes (valores próprios, vetores próprios) com valores próprios decrescentes e
        autovetor[i] correspondente ao autovalor[i]. Os valores próprios devem ter o
        forma (K,) e autovetores devem ter a forma (K, D) com K = min(N, D).
    """
    assert len(X.shape) == 2
    N, D = X.shape
    
    # SEU CÓDIGO AQUI
    
    raise NotImplementedError()

**d)** Efetue os testes de validação abaixo:

In [ ]:
gen = np.random.RandomState(seed=123)
N, D = 4, 3
X = gen.uniform(size=(N, D))
X -= np.mean(X, axis=0)
evals, evecs = svdsolve(X)
C = np.dot(X.T, X) / (N - 1)

assert np.allclose(C, evecs.T.dot(np.diag(evals).dot(evecs)))
assert np.allclose(
    np.round(evals, 5),
    [ 0.08825,  0.0481 ,  0.01983])
assert np.allclose(
    np.round(evecs, 3),
    [[-0.787, -0.477,  0.391],
     [ 0.117, -0.737, -0.665],
     [ 0.606, -0.478,  0.636]])

################################

N, D = 3, 4
X = gen.uniform(size=(N, D))
X -= np.mean(X, axis=0)
evals, evecs = svdsolve(X)
C = np.dot(X.T, X) / (N - 1)

assert np.allclose(C, evecs.T.dot(np.diag(evals).dot(evecs)))
assert np.allclose(
    np.round(evals, 5),
    [ 0.23688,  0.03412,  0.     ])
assert np.allclose(
    np.round(evecs, 3),
    [[ 0.368,  0.874,  0.316, -0.041],
     [ 0.752, -0.178, -0.313,  0.553],
     [-0.475,  0.445, -0.62 ,  0.439]])

**e)** Observe que os autovetores encontrados pelos dois métodos podem diferir por um sinal geral, mas ambos os conjuntos de autovetores são ortonormais, portanto, igualmente válidos.

O código de driver simples a seguir mostra como construir um ajuste de PCA de suas funções (mas o [driver `sklearn`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) tem muito mais opções):

In [ ]:
def PCA_fit(data, n_components = 2):
    X = data.values
    N, D = X.shape
    print('N,D = {},{}'.format(N, D))
    K = min(N, D)
    assert n_components <= K
    
    # Subtraia o valor médio de cada recurso.
    mu = np.mean(X, axis=0)
    Xc = X - mu
    
    # Selecione o método baseado em N, D.
    if N > 2 * D:
        print('Usando o método 1')
        evals, M = eigensolve(np.dot(Xc.T, Xc) / (N - 1))
        assert evals.shape == (D,) and M.shape == (D, D)
    elif D > 2 * N:
        print('Usando o méotodo 2')
        evals, M = eigensolve(np.dot(Xc, Xc.T) / (N - 1))
        assert evals.shape == (N,) and M.shape == (N, N)
        # Os autovetores são agora M = U.T do SVD. Converta para M = V.
        # Use abs(evals), pois os menores valores podem ser < 0 devido a erros numéricos.
        M = np.dot(np.dot(np.diag(np.abs(evals) ** -0.5), M), Xc) / np.sqrt(N - 1)
    else:
        print('Usando o método 3')
        evals, M = svdsolve(Xc)
        assert evals.shape == (K,) and M.shape == (K, D)
        
    # Calcule Y tal que Xc = Y M.
    Y = np.dot(Xc, M.T)
    
    # Aparar para o subespaço de variável latente.
    Y = Y[:, :n_components]
    M = M[:n_components]
    
    # Calcular amostras reconstruídas.
    Xr = np.dot(Y, M) + mu
    
    # Plote algumas amostras e suas reconstruções.
    for i,c in zip((0, 6, 7), sns.color_palette()):
        plt.plot(X[i], '.', c=c, ms=5)
        plt.plot(Xr[i], '-', c=c)
    plt.show()

**f)** Teste este driver em cada regime variando o número de recursos usados de spectra_data com $N$, $D = 200, 500$:

- $D>>D$: Método 1
- $N<<D$: Método 2
- $N\approxeq D$: Método 3

In [ ]:
PCA_fit(spectra_data.iloc[:, 190:230], n_components=2)

In [ ]:
PCA_fit(spectra_data, n_components=2)

In [ ]:
PCA_fit(spectra_data.iloc[:, 120:320], n_components=2)

---